<a href="https://colab.research.google.com/github/PavanPasidu/ScriboSense/blob/saadha/wording_nb%5Cwording1_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
from sklearn.model_selection import train_test_split


In [51]:
import pandas as pd

dataset_path = '/content/drive/My Drive/Student Summarizer/dataset/'

# Read CSV file
summaries = pd.read_csv(dataset_path + 'summaries_train.csv')

# Display the first few rows of the dataset
print(summaries.head())


     student_id prompt_id                                               text  \
0  000e8c3c7ddb    814d6b  The third wave was an experimentto see how peo...   
1  0020ae56ffbf    ebad26  They would rub it up with soda to make the sme...   
2  004e978e639e    3b9047  In Egypt, there were many occupations and soci...   
3  005ab0199905    3b9047  The highest class was Pharaohs these people we...   
4  0070c9e7af47    814d6b  The Third Wave developed  rapidly because the ...   

    content   wording  
0  0.205683  0.380538  
1 -0.548304  0.506755  
2  3.128928  4.231226  
3 -0.210614 -0.471415  
4  3.272894  3.219757  


In [52]:
df=summaries
df.head()


,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


In [53]:
duplicate_rows = df[df.duplicated()]

# Display the duplicate rows
print("Duplicate Rows:" , str(len(duplicate_rows)))

Duplicate Rows: 0


In [54]:
columns_to_drop = ['student_id', 'prompt_id','content']
df = df.drop(columns=columns_to_drop)

df.head()

,text,wording
0,The third wave was an experimentto see how peo...,0.380538
1,They would rub it up with soda to make the sme...,0.506755
2,"In Egypt, there were many occupations and soci...",4.231226
3,The highest class was Pharaohs these people we...,-0.471415
4,The Third Wave developed rapidly because the ...,3.219757


In [55]:
! pip install transformers

In [56]:
import sys
import numpy as np

import torch
from transformers import BertTokenizer, BertForMaskedLM

# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-large-cased')
model.eval()

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')

def score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tokenize_input = ["[CLS]"] + tokenize_input + ["[SEP]"]

    # Convert tokens to IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokenize_input)

    # Convert IDs to tensors
    tensor_input = torch.tensor([token_ids])

    with torch.no_grad():
        # Use the model's MLM forward method
        prediction_scores = model(tensor_input)[0]

    # Calculate the mean loss for the masked tokens
    masked_lm_loss = torch.nn.CrossEntropyLoss(reduction='mean')(prediction_scores.view(-1, tokenizer.vocab_size), tensor_input.view(-1))

    return np.exp(masked_lm_loss.item())

# Test the score function
sentence = "The third wave was an experiment to see how people"
grammar_score = score(sentence)
print("Grammar Score:", grammar_score)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Grammar Score: 37.05259184106069


In [57]:
# Test the score function
sentence = "Some."
grammar_score = score(sentence)
print("Grammar Score:", grammar_score)

Grammar Score: 27878.842859197193


In [ ]:
grammar_scores = []

for sentence in df['text']:
    grammar_score = score(sentence)
    grammar_scores.append(grammar_score)

df['grammar_score'] = grammar_scores

In [ ]:
X = df[['grammar_score']]
y = df['wording']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

train_data = X_train.copy()
train_data['wording'] = y_train['wording']

sns.scatterplot(data=train_data, x='grammar_score', y='wording')
plt.show()

correlation = X_train['grammar_score'].corr(y_train['wording'])
print("Correlation between grammar score and wording:", correlation)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)